In [7]:
using StatsPlots, Statistics, Distributions, Random

In [1]:
function generate_midpoints(n_bins)
    quantiles = Array{Float64}(undef, n_bins)
    quantile_midpoints = Array{Float64}(undef, n_bins)

    #generate the quantile midpoints
    for i = 1:n_bins
        quantiles[i] = i/n_bins
        quantile_midpoints[i] = i/n_bins - (1/2)*(1/n_bins)
    end

    return vec(quantiles), vec(quantile_midpoints)
end

generate_midpoints (generic function with 1 method)

In [2]:
function multiply_distributions(list1, list2)
    sum_list = Array{Float64}(undef, length(list1)^2)
    
    list1 = repeat(list1, length(list1))
    list1 = sort(list1)
    list2 = repeat(list2, length(list2))
    
    for i in eachindex(sum_list)
        sum_list[i] = list1[i] * list2[i]
    end
    
    return vec(sort(sum_list))
end

multiply_distributions (generic function with 1 method)

In [3]:
function re_bin(distribution, n_bins)
    rebinned = reshape(distribution, n_bins, n_bins)
    rebinned = mean(rebinned, dims = 1)
    
    return vec(rebinned)
end

re_bin (generic function with 1 method)

In [4]:
function calculate_mean(RV, n_bins)
    return sum(RV .* 1/n_bins)
end

calculate_mean (generic function with 1 method)

In [5]:
function calculate_variance(RV, calculated_mean, n_bins)
    return sum(RV.^2 * (1/n_bins)) - calculated_mean^2
end

calculate_variance (generic function with 1 method)

In [8]:
n_bins = 20
n_RVs = 30
μ = 6
σ = 3

mean_prod = 1
var_prod = 1

RV_list = Array{Distribution}(undef, n_RVs)

for i = 1:n_RVs
#     μ = rand(5:10)
#     σ = rand(1:6)
    
    temp_RV = Normal(μ, σ)
    RV_list[i] = temp_RV
    
    mean_prod = mean_prod * μ
    var_prod = var_prod * σ^2
end

In [9]:
quantiles, quantile_boundaries = generate_midpoints(n_bins)

final_RV = pop!(RV_list)
final_RV = quantile.(final_RV, quantile_boundaries)

for RV in RV_list
    temp_list = quantile.(RV, quantile_boundaries)
    
    final_RV = multiply_distributions(temp_list, final_RV)
    final_RV = re_bin(final_RV, n_bins) 
end

In [11]:
calc_mean = calculate_mean(final_RV, n_bins)
calc_var = calculate_variance(final_RV, calc_mean, n_bins)

println("Expected Additive Mean is: ", mean_prod)
println("Calculated Additive Mean is: ", calc_mean)

println("Expected Additive Variance is: ", var_prod)
println("Calculated Additive Variance is: ", calc_var)

Expected Additive Mean is: 8138741398091333632
Calculated Additive Mean is: 2.21073919720733e23
Expected Additive Variance is: -3535985420588157519
Calculated Additive Variance is: 6.83246538718013e47
